#  Unit 2.4a Hacks
> Using OOP programming style with CPT database
- toc: true
- image: /images/python.png
- categories: [python]

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- Focus for the lesson will be objects, they have data and depth

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object: Flask is a template for a class. The app is an object in the class.
    2. SQLAlchemy db object: SQLAlchemy uses the template from app to create a db. 


In [ ]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()

# Breakpoint means that we want to examine the things which have occurred before the breakpoint
# Flask helps to create the app object
# app.config adds to the app, since they are defined and assigned to it, can be called setters

# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User: defining the template the class of Users, class definition template, helps us create objects which are of the type User
    - db.Model inheritance: inherit the ability of the class to be able to use database properties, attributes, and methods
    - _init_ method: can be called the constructor, used to instantiate an object from the User class
    - ```@property```, ```@<column>.setter```
    - setters enable us to change the information of the object
    - getters enable us to get the information of the object
    - create, read, update, delete methods: methods which allow us to be able to alter the database, are executable elements whicha re defined by the class, classes can have attributes and methods

In [ ]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
# defining the template the class of Users, class definition template, helps us create objects which are of the type User, template for future object
class Player(db.Model):
    __tablename__ = 'playerstats'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=True, nullable=False)
    _atts = db.Column(db.String(255), unique=False, nullable=False)
    _comps = db.Column(db.String(255), unique=False, nullable=False)
    _yards = db.Column(db.String(255), unique=False, nullable=False)
    _tds = db.Column(db.String(255), unique=False, nullable=False)
    _pimage = db.Column(db.String, unique=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, atts, comps, yards, tds, pimage):
        self._name = name    # variables with self prefix become part of the object, 
        self._atts = atts
        self._comps = comps
        self._yards = yards
        self._tds = tds
        self._pimage = pimage

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    @property
    def atts(self):
        return self._atts
    
    # a setter function, allows name to be updated after initial object creation
    @atts.setter
    def atts(self, atts):
        self._atts = atts

    @property
    def comps(self):
        return self._comps
    
    # a setter function, allows name to be updated after initial object creation
    @comps.setter
    def comps(self, comps):
        self._comps = comps

    @property
    def yards(self):
        return self._yards
    
    # a setter function, allows name to be updated after initial object creation
    @yards.setter
    def yards(self, yards):
        self._yards = yards

    @property
    def tds(self):
        return self._tds
    
    # a setter function, allows name to be updated after initial object creation
    @tds.setter
    def tds(self, tds):
        self._tds = tds

    @property
    def pimage(self):
        return self._pimage
    
    @pimage.setter
    def pimage(self, pimage):
        self._pimage = pimage
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name" : self.name,
            "atts" : self.atts,
            "comps" : self.comps,
            "yards" : self.yards,
            "tds": self.tds,
            "pimage": self.pimage
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name, atts, comps, yards, tds, pimage):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(atts) > 0:
            self.atts = atts
        if len(comps) > 0:
            self.comps = comps
        if len(yards) > 0:
            self.yards = yards
        if len(tds) > 0:
            self.tds = tds
        if len(pimage) > 0:
            self.pimage = pimage
        db.session.add(self)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object: The Create All Tables from db Object creates the users from all of the defined variables in the users list.
    2. User Object Constructors: The User Object Constructors create the rows in the database using the given information from the code.
    3. Try / Except: The try commands attempts to create a new addition to the database. The except command runs if there is an exception in the database.


In [ ]:
"""Database Creation and Testing """


# Builds working data for testing
def initPlayers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        p1 = Player(name='Patrick Mahomes', atts='648', comps='435', yards='5250', tds='41', pimage='images/pm.png')
        p2 = Player(name='Justin Herbert', atts='699', comps='477', yards='4739', tds='25', pimage='images/jh.png')
        p3 = Player(name='Tom Brady', atts='733', comps='490', yards='4694', tds='25', pimage='images/tb.png')
        p4 = Player(name='Kirk Cousins', atts='643', comps='424', yards='4547', tds='29', pimage='images/kc.png')
        p5 = Player(name='Joe Burrow', atts='606', comps='414', yards='4475', tds='35', pimage='images/jb.png')
        p6 = Player(name='Jared Goff', atts='587', comps='382', yards='4438', tds='29', pimage='images/jg.png')
        p7 = Player(name='Josh Allen', atts='567', comps='359', yards='4283', tds='35', pimage='images/ja.png')
        p8 = Player(name='Geno Smith', atts='572', comps='399', yards='4283', tds='30', pimage='images/gs.png')
        p9 = Player(name='Trevor Lawrence', atts='584', comps='387', yards='4113', tds='25', pimage='images/tl.png')
        p10 = Player(name='Jalen Hurts', atts='460', comps='306', yards='3701', tds='22', pimage='images/jhurts.png')
        qbs = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]
        for qb in qbs:
            try:
                object = qb.create()
                print(f"Added data for {object.name}")
            except IntegrityError:
                '''fails with bad or duplicate data'''
                print(f"Records exist, duplicate email, or error: {qb.name}")

initPlayers()

Added data for Patrick Mahomes
Added data for Justin Herbert
Added data for Tom Brady
Added data for Kirk Cousins
Added data for Joe Burrow
Added data for Jared Goff
Added data for Josh Allen
Added data for Geno Smith
Added data for Trevor Lawrence
Added data for Jalen Hurts


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by: the User.query.filter_by searches for the specific user or users. In this case, using User.query.filter_by finds the first user and assigns it to a variable.
    2. user.password: checks and references the password of the user. 

In [ ]:
def find_by_name(name):
    with app.app_context():
        # ORM allows us to do query methods on our data
        # only returns the match to the first match to the database
        player = Player.query.filter_by(_name=name).first()
    return player # returns user object

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except: user.find_by_uid() checks if they uid exists. Try will run if the uid is found and except will run if it is not found.
    2. user = User(...): this defines the current parameters of User, so everything at the point is part of user.
    3. user.dob and try/except: user.dob is a function which converts the input from the consumer into a dob string which is valid for storage. If it meets the requirements for storage, try is run. Except is run if there is an error in the inputs.
    4. user.create() and try/except: user.create() runs the create() command to create a user. The try portion will output created if it is met. The except portion will output an error if it does not work.

In [ ]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    name = input("Enter a QB name:")
    pyer = find_by_name(name)
    try:
        print("Found\n", pyer.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    atts = input("Enter their attempts:")
    comps = input("Enter their completions:")
    yards = input("Enter their yards:")
    tds = input("Enter their tds:")
    pimage = input("Enter a link to their image:")

    # Initialize User object before date
    player = Player(name=name, 
                atts=atts, 
                comps=comps,
                yards=yards,
                tds=tds,
                pimage=pimage
                )
           
    # write object to database
    with app.app_context():
        try:
            object = player.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {name}")
        
create()

Created
 {'id': 11, 'name': '', 'atts': '', 'comps': '', 'yards': '', 'tds': '', 'pimage': ''}


In [ ]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Player.query.all()
    json_ready = [player.read() for player in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Patrick Mahomes',
  'atts': '648',
  'comps': '435',
  'yards': '5250',
  'tds': '41',
  'pimage': 'images/pm.png'},
 {'id': 2,
  'name': 'Justin Herbert',
  'atts': '699',
  'comps': '477',
  'yards': '4739',
  'tds': '25',
  'pimage': 'images/jh.png'},
 {'id': 3,
  'name': 'Tom Brady',
  'atts': '733',
  'comps': '490',
  'yards': '4694',
  'tds': '25',
  'pimage': 'images/tb.png'},
 {'id': 4,
  'name': 'Kirk Cousins',
  'atts': '643',
  'comps': '424',
  'yards': '4547',
  'tds': '29',
  'pimage': 'images/kc.png'},
 {'id': 5,
  'name': 'Joe Burrow',
  'atts': '606',
  'comps': '414',
  'yards': '4475',
  'tds': '35',
  'pimage': 'images/jb.png'},
 {'id': 6,
  'name': 'Jared Goff',
  'atts': '587',
  'comps': '382',
  'yards': '4438',
  'tds': '29',
  'pimage': 'images/jg.png'},
 {'id': 7,
  'name': 'Josh Allen',
  'atts': '567',
  'comps': '359',
  'yards': '4283',
  'tds': '35',
  'pimage': 'images/ja.png'},
 {'id': 8,
  'name': 'Geno Smith',
  'atts': '572',


In [ ]:
def updateq():    
    name = input("Enter the name of the QB to update: ")
    pyer = find_by_name(name)
    
    # Request new values for the QB attributes
    atts = input("Enter their attempted passes: ")
    comps = input("Enter their completed passes: ")
    yards = input("Enter their yards this season: ")
    tds = input("Enter how many touchdowns they have thrown for: ")
    pimage = input("Enter their image link: ")
    
    if pyer is not None:
        with app.app_context():
            pyer.update(name=name, atts=atts, comps=comps, yards=yards, tds=tds, pimage=pimage)
            print("Updated Player", name)
    else:
        print("error")

updateq()

Updated Player Patrick Mahomes


In [ ]:
def delete():
    # optimize user time to see if uid exists
    name = input("Enter a QB name:")
    pyer = find_by_name(name)
    try:
        pass
    except:
        name = input("Try again, that was not a valid name:")
    
    with app.app_context():
        try:
            object = pyer.delete()
            print("Deleted\n", pyer)
        except:  # error raised if object not created
            print("Unknown error uid {name}")
delete()

Deleted
 {"id": 11, "name": "", "atts": "", "comps": "", "yards": "", "tds": "", "pimage": ""}


In [ ]:
def crudmenu():
    selection = input("Enter a letter to select your option --> c: Create, r: Read, u: Update, d: delete")
    if selection.lower() == "c":
        create()
    elif selection.lower() == "r":
        with app.app_context():
            table = Player.query.all()
        json_ready = [player.read() for player in table]
        return json_ready
    elif selection.lower() == "u":
        updateq()
    elif selection.lower() == "d":
        delete()
    else:
        selection = input("Please enter a valid letter --> c: Create, r: Read, u: Update, D: delete")

crudmenu()

Deleted
 {"id": 11, "name": "Joe Montana", "atts": "12", "comps": "12", "yards": "12", "tds": "12", "pimage": "12"}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all: user.query.all goes through the entire list of the users and makes sure that all rows of the SQLAlchemy database are selected.
    2. json_ready assignment, google List Comprehension: the json_ready assignment reads all of the data in the SQLAlchemy table. List comprehension means reducing the running of a for loop to one line of code.

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.